In [53]:
import sys
sys.path.append('../src')
import os
import code3_data_ as f3


import requests
import json
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import geopandas as gpd

import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry

In [2]:
load_dotenv()

True

In [3]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [4]:
df5 = f3.import_csv("../data_processed/dfcompanies1_v2.csv")

In [5]:
df5

,name,category_code,city,country_code,latitude,longitude
0,plista,advertising,Berlin,DEU,52.528361,13.412456
1,99designs,design,Berlin,DEU,52.498620,13.446903
2,babbel,education,Berlin,DEU,52.489700,13.388610
3,Deezer,music,Paris,FRA,48.878060,2.328385
4,Adknowledge,advertising,Paris,FRA,48.850606,2.379783
5,Netsize,mobile,Paris,FRA,48.894312,2.288319
6,Solairedirect,cleantech,Paris,FRA,48.874595,2.321074
7,2 Minutes,games_video,Paris,FRA,48.850206,2.369803
8,Bomgar,software,Paris,FRA,48.875231,2.298337
9,Viadeo,social,Paris,FRA,48.874706,2.330416


In [6]:
unique_cities = list(df5.city.unique())

In [7]:
unique_cities

['Berlin', 'Paris', 'Amsterdam']

## Requirements for location (part II)

In [8]:
def req_from_foursq(city,criteria):

    pt_ = f3.get_point(f"{city}")
    
    url= 'https://api.foursquare.com/v2/venues/search'

    params = {
        "client_id" : tok1,
        "client_secret" : tok2,
        "v": "20200210",
        "ll": f"{pt_.get('coordinates')[1]},{pt_.get('coordinates')[0]}",
        "radius":2000,
        "limit": 6,
        "categoryId":f"{str(criteria)}"
    }
    
    response = requests.get(url,params)
    resp = json.loads(response.text)
    
    blob = resp["response"]["venues"]
    lst = []
    for each in blob:
        requirement = each["categories"][0]["name"]
        name = each['name']
        lat = each["location"]["lat"]
        long = each["location"]["lng"]
        req_name_lat_long = {"requirement": requirement, "name": name, "lat": lat, "long": long}
        lst.append(req_name_lat_long)
        
    return lst
        


In [9]:
pre = '52e81612bcbc57f1066b7a45'
private = '52e81612bcbc57f1066b7a46'
middle = '4f4533814b9074f6e4fb0106'
elem = '4f4533804b9074f6e4fb0105'
cocktail_bar = '4bf58dd8d48988d11e941735'
music_venue = '4bf58dd8d48988d1e5931735'

criteria_airport = '4bf58dd8d48988d1ed931735'

criteria_schools = pre + "," + private + "," + middle + ","+ elem

criteria_party = cocktail_bar + "," + music_venue

criteria_food = '4bf58dd8d48988d1d3941735'

### Berlin

In [10]:
city = "Berlin"

In [11]:
schools = req_from_foursq(city,criteria_schools)

In [12]:
airport = req_from_foursq(city,criteria_airport)

In [13]:
party = req_from_foursq(city,criteria_party)

In [14]:
food = req_from_foursq(city,criteria_food)

In [15]:
dfbs = pd.DataFrame(schools)
dfba = pd.DataFrame(airport)
dfbp = pd.DataFrame(party)
dfbf = pd.DataFrame(food)
dfberlin = pd.concat([dfbs,dfba,dfbp,dfbf],ignore_index=True)

In [16]:
dfberlin

,requirement,name,lat,long
0,Elementary School,Lemgo Grundschule,52.494091,13.413214
1,Elementary School,GutsMuths-Grundschule,52.516152,13.424436
2,Elementary School,Grundschule am Brandenburger Tor,52.512810,13.382686
3,Elementary School,Blumen-Grundschule,52.515183,13.432307
4,Middle School,Schule der Russischen Botschaft,52.515294,13.384667
5,Daycare,Kita 1,52.485876,13.391435
6,Airport Lounge,Melli Beese Lounge,52.480565,13.388803
7,Cocktail Bar,Satellite Cocktail,52.493619,13.394989
8,Music Venue,Huxleys Neue Welt,52.486593,13.421428
9,Nightclub,Gretchen,52.496089,13.387626


### Paris

In [17]:
city = "Paris"

In [18]:
pt_ = f3.get_point(f"{city}")

In [19]:
schools = req_from_foursq(city,criteria_schools)
airport = req_from_foursq(city,criteria_airport)
party = req_from_foursq(city,criteria_party)
food = req_from_foursq(city,criteria_food)

In [20]:
dfps = pd.DataFrame(schools)
dfpa = pd.DataFrame(airport)
dfpp = pd.DataFrame(party)
dfpf = pd.DataFrame(food)
dfparis = pd.concat([dfps,dfpa,dfpp,dfpf],ignore_index=True)

In [21]:
dfparis

,requirement,name,lat,long
0,Elementary School,Ecole des Quatre Fils,48.860392,2.360192
1,Elementary School,École Cambon,48.868087,2.326922
2,Elementary School,École Élémentaire Delambre,48.841615,2.326762
3,Elementary School,Ecole élémentaire Chomel,48.852166,2.325719
4,Elementary School,École élémentaire Saint Sébastien,48.861402,2.370036
5,Middle School,Collège Stanislas,48.845282,2.328318
6,Airport Service,Bus Naar Michelbeke,48.863836,2.354640
7,Travel Agency,Agence Air France Invalides,48.862407,2.314740
8,Sports Bar,La Lucha Libre,48.848999,2.348624
9,Cocktail Bar,Bar Joséphine,48.851065,2.327131


### Amsterdam

In [22]:
city = "Amsterdam"

In [23]:
pt_= f3.get_point(f"{city}")
pt_

{'type': 'Point', 'coordinates': [4.89668, 52.36053]}

In [24]:
schools = req_from_foursq(city,criteria_schools)
airport = req_from_foursq(city,criteria_airport)
party = req_from_foursq(city,criteria_party)
food = req_from_foursq(city,criteria_food)

In [25]:
dfas = pd.DataFrame(schools)
dfaa = pd.DataFrame(airport)
dfap = pd.DataFrame(party)
dfaf = pd.DataFrame(food)
dfamsterdam = pd.concat([dfas,dfaa,dfap,dfaf],ignore_index=True)

In [26]:
dfamsterdam

,requirement,name,lat,long
0,Elementary School,De springstok o.b.s,52.356108,4.902459
1,Elementary School,OBS De Burght,52.378357,4.890504
2,Elementary School,Theo Thijssenschool,52.376460,4.881843
3,Elementary School,Dr Rijk Kramerschool,52.376178,4.875026
4,Elementary School,Alan Türingschool,52.370799,4.920787
5,Elementary School,De Witte Olifant,52.370622,4.904049
6,Airport Terminal,Pyong Yang,52.354171,4.887278
7,Airport Lounge,Schiphol lounge,52.353442,4.890861
8,Bar,Freddy's Bar,52.367605,4.894083
9,BBQ Joint,Jack’s BBQ Shack,52.366074,4.881010


### Creating new collections with 2dsphere indexes in ironhack database:   
####    ` hqreq_berlin`, ` hqreq_paris`,  `hqreq_amsterdam`, `companies1`


In [27]:
client = MongoClient()

In [28]:
client = MongoClient()
hqreq_berlin = client.ironhack.hqreq_berlin
hqreq_paris = client.ironhack.hqreq_paris
hqreq_amsterdam = client.ironhack.hqreq_amsterdam
companies1 = client.ironhack.companies1

In [29]:
hqreq_berlin.create_index([("geometry",GEOSPHERE)])
hqreq_paris.create_index([("geometry",GEOSPHERE)])
hqreq_amsterdam.create_index([("geometry",GEOSPHERE)])
companies1.create_index([("geometry",GEOSPHERE)])

'geometry_2dsphere'

In [36]:
df5

,name,category_code,city,country_code,latitude,longitude
0,plista,advertising,Berlin,DEU,52.528361,13.412456
1,99designs,design,Berlin,DEU,52.498620,13.446903
2,babbel,education,Berlin,DEU,52.489700,13.388610
3,Deezer,music,Paris,FRA,48.878060,2.328385
4,Adknowledge,advertising,Paris,FRA,48.850606,2.379783
5,Netsize,mobile,Paris,FRA,48.894312,2.288319
6,Solairedirect,cleantech,Paris,FRA,48.874595,2.321074
7,2 Minutes,games_video,Paris,FRA,48.850206,2.369803
8,Bomgar,software,Paris,FRA,48.875231,2.298337
9,Viadeo,social,Paris,FRA,48.874706,2.330416


### Converting to geopandas dataframe with shapely point objects

In [31]:
gdfberlin = gpd.GeoDataFrame(dfberlin, geometry = gpd.points_from_xy(dfberlin.long, dfberlin.lat))

In [34]:
gdfparis = gpd.GeoDataFrame(dfparis, geometry = gpd.points_from_xy(dfparis.long, dfparis.lat))

In [35]:
gdfamsterdam = gpd.GeoDataFrame(dfamsterdam, geometry = gpd.points_from_xy(dfamsterdam.long, dfamsterdam.lat))

In [37]:
gdf5 = gpd.GeoDataFrame(df5, geometry = gpd.points_from_xy(df5.longitude, df5.latitude))

<div class="girk">
from shapely import wkt</div><i class="fa fa-lightbulb-o "></i>

<div class="mark">
gdf5['geometry']=gpd.GeoSeries.from_wkt(gdf5['geometry'])</div><i class="fa fa-lightbulb-o "></i>

### Converting shapely point objects to GeoJSON point objects

In [41]:
gdfberlin['geometry']=gdfberlin['geometry'].apply(lambda x: shapely.geometry.mapping(x))
gdfparis['geometry']= gdfparis['geometry'].apply(lambda x: shapely.geometry.mapping(x))
gdfamsterdam['geometry']= gdfamsterdam['geometry'].apply(lambda x: shapely.geometry.mapping(x))

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [62]:
df5

,name,category_code,city,country_code,latitude,longitude,geometry
0,plista,advertising,Berlin,DEU,52.528361,13.412456,POINT (13.41246 52.52836)
1,99designs,design,Berlin,DEU,52.498620,13.446903,POINT (13.44690 52.49862)
2,babbel,education,Berlin,DEU,52.489700,13.388610,POINT (13.38861 52.48970)
3,Deezer,music,Paris,FRA,48.878060,2.328385,POINT (2.32839 48.87806)
4,Adknowledge,advertising,Paris,FRA,48.850606,2.379783,POINT (2.37978 48.85061)
5,Netsize,mobile,Paris,FRA,48.894312,2.288319,POINT (2.28832 48.89431)
6,Solairedirect,cleantech,Paris,FRA,48.874595,2.321074,POINT (2.32107 48.87460)
7,2 Minutes,games_video,Paris,FRA,48.850206,2.369803,POINT (2.36980 48.85021)
8,Bomgar,software,Paris,FRA,48.875231,2.298337,POINT (2.29834 48.87523)
9,Viadeo,social,Paris,FRA,48.874706,2.330416,POINT (2.33042 48.87471)


In [ ]:
gdf5['geometry']=gdf5['geometry'].apply(lambda x: shapely.geometry.mapping(x))

In [47]:
gdf5

,name,category_code,city,country_code,latitude,longitude,geometry
0,plista,advertising,Berlin,DEU,52.528361,13.412456,POINT (13.41246 52.52836)
1,99designs,design,Berlin,DEU,52.498620,13.446903,POINT (13.44690 52.49862)
2,babbel,education,Berlin,DEU,52.489700,13.388610,POINT (13.38861 52.48970)
3,Deezer,music,Paris,FRA,48.878060,2.328385,POINT (2.32839 48.87806)
4,Adknowledge,advertising,Paris,FRA,48.850606,2.379783,POINT (2.37978 48.85061)
5,Netsize,mobile,Paris,FRA,48.894312,2.288319,POINT (2.28832 48.89431)
6,Solairedirect,cleantech,Paris,FRA,48.874595,2.321074,POINT (2.32107 48.87460)
7,2 Minutes,games_video,Paris,FRA,48.850206,2.369803,POINT (2.36980 48.85021)
8,Bomgar,software,Paris,FRA,48.875231,2.298337,POINT (2.29834 48.87523)
9,Viadeo,social,Paris,FRA,48.874706,2.330416,POINT (2.33042 48.87471)


### Reconverting geoDFs into lists of dictionaries to insert filtered documents with added GeoJSON information into their corresponding collection.

In [ ]:
gdf_dic_berlin = gdfberlin.to_dict('records')
gdf_dic_paris = gdfparis.to_dict('records')
gdf_dic_ams = gdfamsterdam.to_dict('records')

In [61]:
gdf5_dict= gdf5.to_dict('records')
gdf5_dict[0]["geometry"]["type"]

TypeError: 'Point' object is not subscriptable

In [ ]:
hqreq_berlin.insert_many(gdf_dic_berlin)
hqreq_paris.insert_many(gdf_dic_paris)
hqreq_amsterdam.insert_many(gdf_dic_ams)

In [ ]:
companies2.insert_many(gdf5_dict)

In [ ]:
f3.export_df_to_csv(gdfberlin)
f3.export_df_to_csv(gdfparis)
f3.export_df_to_csv(gdfamsterdam)

In [ ]:
f3.export_df_to_csv(gdf5)


In [ ]:
'''
def fetch_coord_geocode(unique_cities):
    geopy = Nominatim(user_agent="cris@callejo.co.uk")
    for city in unique_cities:
        try:
            return geopy.geocode(city,exactly_one=True,addressdetails = 0,timeout=5,language="en",)
            pass
        except GeocoderTimedOut:
            if attempt <= max_attempts:
                return do_geocode(address,attempt=attempt+1)
            raise
'''

In [ ]:
def fetch_coord_geocode(Berlin):
    geopy = Nominatim(user_agent="cris@callejo.co.uk")
    cities_coord_raw = {}
    try:
        for city in unique_cities:
            coord_raw = geopy.geocode(f"{city}",addressdetails = 0,timeout=5,language="en")
            cities_coord_raw[city]= coord_raw 
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return do_geocode(city,attempt=attempt+1)
        raise
        #return geopy.geocode(city,exactly_one=True,addressdetails = 0,timeout=5,language="en",)
    return cities_coord_raw

In [ ]:
cities_coord_raw = fetch_coord_geocode(unique_cities)

In [ ]:
print(cities_coord_raw)
for k,v in cities_coord_raw.items():
    print(v)
    
print(type(cities_coord_raw))

In [ ]:
def fetching_coord(lst):
    dct = {}
    for each in lst:
        url_base = "https://nominatim.openstreetmap.org/search"
        params = f"?q={each}&limit=3&format=geojson&polygon_geojson=1"
        response = requests.get(url_base+params)
        resp = response.json()
        dct[each] = resp
    return dct

In [ ]:
fetching_coord(unique_cities)
# berlin multipolygon second option
#paris polygon segunda
# ams multipolygon primera o tercera

In [ ]:
dct = fetching_coord(unique_cities)


In [ ]:
{'Berlin': 
 {'type': 'FeatureCollection',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'features': 
      [
      {'type': 'Feature',
        'properties': 
           {'place_id': 566458,
             'osm_type': 'node',
             'osm_id': 240109189,
             'display_name': 'Berlin, 10117, Deutschland',
             'place_rank': 15,
             'category': 'place',
             'type': 'city',
             'importance': 0.8975390282491362,
             'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png'},
            'bbox': [13.2288599, 52.3570365, 13.5488599, 52.6770365],
            'geometry': 
                   {'type': 'Point', 'coordinates': [13.3888599, 52.5170365]}},
       {'type': 'Feature',
    'properties': {'place_id': 256873689,
     'osm_type': 'relation',
     'osm_id': 62422,
     'display_name': 'Berlin, Deutschland',
     'place_rank': 8,
     'category': 'boundary',
     'type': 'administrative',
     'importance': 0.8975390282491362,
     'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'},
    'bbox': [13.088345, 52.3382448, 13.7611609, 52.6755087],
    'geometry': {'type': 'MultiPolygon',

In [ ]:
#coord_ber = dct["Berlin"]["features"][0]["geometry"]
coord_ber = dct.select("MultiPolygon")
#["Berlin"]
#["type"]
#["features"][0]["geometry"]
coord_ber

In [ ]:
coord_ber = ["Berlin"]["type":"Feature"]
coord_ber

In [ ]:
coord_ber = dct["Berlin"]["features"][0][1]["geometry"]

coord_par = dct["Paris"]["features"][0]["geometry"]

coord_ams = dct["Amsterdam"]["features"][0]["geometry"]

cities_dct = {"Berlin": coord_ber, "Paris": coord_par, "Amsterdam": coord_ams}

In [ ]:
dct["Berlin"]["features"][0]["Polygon"]

In [ ]:
dct["Paris"]

In [ ]:
dct["Amsterdam"]

In [ ]:
dct.keys()

In [ ]:
geo_spania

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-4.893577, 43.238479],
       [-4.839201, 43.18976]]]]}

 'properties': {'cod_ccaa': '07',
    'noml_ccaa': 'COMUNIDAD DE CASTILLA Y LEON',
    'name': 'Castilla-Leon',
    'cartodb_id': 7,
    'created_at': '2014-09-30T00:00:00Z',
    'updated_at': '2014-12-25T02:07:41Z'}},
{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[0.716037, 42.858321],

In [ ]:
dct['type']['features']